$$\Large \textit{C0708 | Reconocimiento de Patrones}$$

$$\large \textbf{Challenge Nº3 | Máquina de Soporte Vectorial}$$



- _Alessandra Mercedes Aldave Javier_

#1. Introducción

Los sideroforos son unos metabolitos secundarios necesarios que algunos organismos sintetizan para poder captar hierro de su entorno [1]. Estos metabolitos presentan una alta afinidad y especificidad por el hierro como ligandos organicos. No obstante, estas moleculas tienen un rol que va mas alla de ser transportadores de hierro, sino que tambien participan como mediadores de interacciones con otros microorganismos y con eucariontes.[2]

#2. Metodología

A continuacion se describen en comentarios los pasos seguidos para llegar a la clasificacion incluyendo el preprocesamiento de la data.

In [181]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
#from sklearn.multiclass import OneVsOneClassifier
%matplotlib inline

In [182]:
#Realizamos lectura de los data frames a utilizar

#Primero del target
df_target = pd.read_csv("https://raw.githubusercontent.com/inefable12/siderophores_database/main/Siderophore_DB.csv",index_col=0)
df_target

,Compounds,Microorganism,hydroxamate,catecholate,a-hydroxycarboxylate,carboxylate,phenolate,citrate,other,SMILES,validate smiles,SMARTS,SELFIES
ID,,,,,,,,,,,,,
1,"2,3-dihydroxybenzoic acid","Azobacter vinelandii, Azospirillum lipoferum, ...",0,1,0,1,0,0,0,OC(=O)c1c(O)c(O)ccc1,True,[#8]-[#6](=[#8])-[#6]1:[#6](-[#8]):[#6](-[#8])...,[O][C][=Branch1][C][=O][C][=C][Branch1][C][O][...
2,"2,3-dihydroxybenzoylglycine",Bacillus subtilis,0,1,0,1,0,0,0,c1ccc(O)c(O)c1C(=O)NCC(O)=O,True,[#6]1:[#6]:[#6]:[#6](-[#8]):[#6](-[#8]):[#6]:1...,[C][=C][C][=C][Branch1][C][O][C][Branch1][C][O...
3,"2,3-dihydroxybenzoylserine","Aerobacter aerogenes, Escherichia ST2, Escheri...",0,1,0,1,0,0,0,OC[C@@H](C(O)=O)NC(=O)c1c(O)c(O)ccc1,True,[#8]-[#6]-[#6@@H](-[#6](-[#8])=[#8])-[#7]-[#6]...,[O][C][C@@H1][Branch1][=Branch1][C][Branch1][C...
4,"2,3-dihydroxybenzoylthreonine",Klebsiella oxytoca,0,1,0,1,0,0,0,O[C@H](C)[C@@H](C(O)=O)NC(=O)c1cccc(O)c1O,True,[#8]-[#6@H](-[#6])-[#6@@H](-[#6](-[#8])=[#8])-...,[O][C@H1][Branch1][C][C][C@@H1][Branch1][=Bran...
5,2-hydroxybenzoic acid,"Azospirillum lipoferum, Burkholderia cepacia",0,0,0,1,1,0,0,OC(=O)c1c(O)cccc1,True,[#8]-[#6](=[#8])-[#6]1:[#6](-[#8]):[#6]:[#6]:[...,[O][C][=Branch1][C][=O][C][=C][Branch1][C][O][...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,Desferriastechrome,NaN,2,0,0,0,0,0,0,O=c1c(C)nc(OC)c(n1O)Cc2c[nH]c(c23)c(ccc3)CC=C(C)C,True,[#8]=[#6]1:[#6](-[#6]):[#7]:[#6](-[#8]-[#6]):[...,[O][=C][C][Branch1][C][C][=N][C][Branch1][Ring...
278,Fetroja,NaN,0,1,0,2,0,0,1,c1cc(O)c(O)c(Cl)c1C(=O)NCC[N+]2(CCCC2)CC(=C3C(...,True,[#6]1:[#6]:[#6](-[#8]):[#6](-[#8]):[#6](-[#17]...,[C][=C][C][Branch1][C][O][=C][Branch1][C][O][C...
279,Cefiderocol,NaN,0,1,0,2,0,0,1,c1cc(O)c(O)c(Cl)c1C(=O)NCC[N+]2(CCCC2)CC(=C3C(...,True,[#6]1:[#6]:[#6](-[#8]):[#6](-[#8]):[#6](-[#17]...,[C][=C][C][Branch1][C][O][=C][Branch1][C][O][C...


In [183]:
#Y luego de los features
df_features = pd.read_csv("https://raw.githubusercontent.com/inefable12/siderophores_database/main/Siderophore_DB1.csv",index_col=0)
df_features

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,MaxPartialCharge,...,fr_NH0,fr_NH1,fr_NH2,fr_N_O,fr_amide,fr_benzene,fr_ester,fr_para_hydroxylation,fr_phenol,fr_phenol_noOrthoHbond
ID,,,,,,,,,,,,,,,,,,,,,
1,11.191262,-1.343472,11.191262,0.665370,0.522491,154.121,148.073,154.026609,58,0.339119,...,0,0,0,0,0,1,0,1,2,1
2,12.228676,-3.436129,12.228676,0.586755,0.522624,211.173,202.101,211.048072,80,0.322481,...,0,1,0,0,1,1,0,1,2,2
3,12.627825,-3.816337,12.627825,0.758838,0.442264,241.199,230.111,241.058637,92,0.328131,...,0,1,0,0,1,1,0,1,2,2
4,12.854441,-4.102795,12.854441,0.884463,0.468346,255.226,242.122,255.074287,98,0.328426,...,0,1,0,0,1,1,0,1,2,2
5,11.108779,-1.310880,11.108779,0.630759,0.610259,138.122,132.074,138.031694,52,0.338969,...,0,0,0,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,13.053345,-3.721588,13.053345,0.123679,0.544261,353.422,330.238,353.173942,136,0.327527,...,2,1,0,0,0,1,0,1,0,0
278,14.672799,-5.563063,14.672799,0.044514,0.049920,753.236,717.956,752.156986,268,0.352359,...,4,2,1,0,3,1,0,0,2,2
279,14.672799,-5.563063,14.672799,0.044514,0.049920,753.236,717.956,752.156986,268,0.352359,...,4,2,1,0,3,1,0,0,2,2


In [184]:
#Tras revisar la data, y para simplificar el proceso de clasificacion, se trabajara unicamente con aquellas moleculas que presenten unica y excluyentemente un grupo funcional
#Eso signfica que desde la columna 4 hasta la columna 10, se revisara cuales son aquellas moleculas que tienen todas sus celdas con valores de cero y solo una celda con cualquier otro numero diferente de cero
#Esas filas compondrán el nuevo dataframe con el que se trabajará

df_booleano = (df_target.iloc[:,2:9] != 0) #Selecciono desde la 4ta hasta la 10a columna del dataframe y hago una comparación de cada celda con el valor 0. Si una celda es diferente de cero, la celda tendra el valor de True, si de lo contrario es igual a cero, el valor de la celda en booleano sera False
#df_booleano.head(25)
df_suma_De_nonzeros = df_booleano.sum(axis=1)             #Sumo todos los valores booleanos de cada fila
#df_suma_De_nonzeros.head(25)
df_target_nuevo = df_target[df_suma_De_nonzeros == 1]             #Obtengo un dataframe en el que la suma de todos los True en una fila sea igual a 1, es decir, en donde solo haya una celda que tenga un numero diferente de cero y ese numero sea igual a la suma de todos los valores de esa fila
#df_target_nuevo.head(25)
#df_target_nuevo
df_target_nuevo.head(25)

,Compounds,Microorganism,hydroxamate,catecholate,a-hydroxycarboxylate,carboxylate,phenolate,citrate,other,SMILES,validate smiles,SMARTS,SELFIES
ID,,,,,,,,,,,,,
6,2-N-methylcoprogen,Magnaporthe grisea,3,0,0,0,0,0,0,OCC\C(C)=C\C(=O)N(O)CCC[C@H](N(C)C(=O)C)C(=O)O...,True,[#8]-[#6]-[#6]/[#6](-[#6])=[#6]/[#6](=[#8])-[#...,[O][C][C][\C][Branch1][C][C][=C][\C][=Branch1]...
6_ChEBI,desferricoprogen B,NaN,3,0,0,0,0,0,0,OCC\C(C)=C\C(=O)N(O)CCC[C@H](N)C(=O)OCCC(\C)=C...,True,[#8]-[#6]-[#6]/[#6](-[#6])=[#6]/[#6](=[#8])-[#...,[O][C][C][\C][Branch1][C][C][=C][\C][=Branch1]...
7,2-N-methylcoprogen B,"Histoplasma capsulatum, Magnaporthe grisea, Ps...",3,0,0,0,0,0,0,OCC\C(C)=C\C(=O)N(O)CCC[C@H](NC)C(=O)OCCC(\C)=...,True,[#8]-[#6]-[#6]/[#6](-[#6])=[#6]/[#6](=[#8])-[#...,[O][C][C][\C][Branch1][C][C][=C][\C][=Branch1]...
11_r1,acinetobactin,"Acinetobacter baumannii, Acinetobacter haemoli...",0,1,0,0,0,0,0,c1nc[nH]c1CCN2O[C@@H](C)[C@@H](C2=O)NC(=O)c3c(...,True,[#6]1:[#7]:[#6]:[#7H]:[#6]:1-[#6]-[#6]-[#7]1-[...,[C][N][=C][NH1][C][=Ring1][Branch1][C][C][N][O...
13,acyl-desferrioxamine 1,Micrococcus luteus,3,0,0,0,0,0,0,c1ccccc1CC(=O)N(O)CCCCCNC(=O)CCC(=O)N(O)CCCCCN...,True,[#6]1:[#6]:[#6]:[#6]:[#6]:[#6]:1-[#6]-[#6](=[#...,[C][=C][C][=C][C][=C][Ring1][=Branch1][C][C][=...
14,acyl-desferrioxamine 2,Micrococcus luteus,3,0,0,0,0,0,0,c1ccccc1CC(=O)N(O)CCCCCNC(=O)CCC(=O)N(O)CCCCCN...,True,[#6]1:[#6]:[#6]:[#6]:[#6]:[#6]:1-[#6]-[#6](=[#...,[C][=C][C][=C][C][=C][Ring1][=Branch1][C][C][=...
015_r1,acyl-desferrioxamine 3,Micrococcus luteus,3,0,0,0,0,0,0,CC[C@H](C)CCCCC(=O)N(O)CCCCCNC(=O)CCC(=O)N(O)C...,True,[#6]-[#6]-[#6@H](-[#6])-[#6]-[#6]-[#6]-[#6]-[#...,[C][C][C@H1][Branch1][C][C][C][C][C][C][C][=Br...
016_r1,acyl-desferrioxamine 4,Micrococcus luteus,3,0,0,0,0,0,0,CC[C@H](C)CCCCCCC(=O)N(O)CCCCCNC(=O)CCC(=O)N(O...,True,[#6]-[#6]-[#6@H](-[#6])-[#6]-[#6]-[#6]-[#6]-[#...,[C][C][C@H1][Branch1][C][C][C][C][C][C][C][C][...
016_r2,acyl-desferrioxamine 4_,Micrococcus luteus,3,0,0,0,0,0,0,CC[C@@H](C)CCCCCCC(=O)N(O)CCCCCNC(=O)CCC(=O)N(...,True,[#6]-[#6]-[#6@@H](-[#6])-[#6]-[#6]-[#6]-[#6]-[...,[C][C][C@@H1][Branch1][C][C][C][C][C][C][C][C]...


In [185]:
#Ahora, se revisará cuales son las categorias con las que finalmente trabajaremos, pues solo 1 de las 7 categorias(por cada fila) tendra un valor diferente a 0 en todas sus celdas.
#Para ello, sumaré por columnas y solo me quedaré con aquellas columnas o categorias cuyas celdas no sumen 0 en total (o sumen una cantidad diferente de cero).
#Considerando que mis categorias o columnas de interes son mis grupos funcionales

grupos_funcionales = df_target_nuevo.iloc[:,2:9]                                 #Selecciono las columnas de interes de todo el dataframe
grupos_funcionales

,hydroxamate,catecholate,a-hydroxycarboxylate,carboxylate,phenolate,citrate,other
ID,,,,,,,
6,3,0,0,0,0,0,0
6_ChEBI,3,0,0,0,0,0,0
7,3,0,0,0,0,0,0
11_r1,0,1,0,0,0,0,0
13,3,0,0,0,0,0,0
...,...,...,...,...,...,...,...
270,3,0,0,0,0,0,0
272,3,0,0,0,0,0,0
273,3,0,0,0,0,0,0


In [186]:
grupos_func_conElementos = grupos_funcionales.columns[grupos_funcionales.sum() !=0] #Obtengo los nombres de aquellas columnas cuya suma de elementos es diferente a cero
df_target_final = df_target_nuevo[grupos_func_conElementos]
df_target_final

,hydroxamate,catecholate,a-hydroxycarboxylate,phenolate,citrate,other
ID,,,,,,
6,3,0,0,0,0,0
6_ChEBI,3,0,0,0,0,0
7,3,0,0,0,0,0
11_r1,0,1,0,0,0,0
13,3,0,0,0,0,0
...,...,...,...,...,...,...
270,3,0,0,0,0,0
272,3,0,0,0,0,0
273,3,0,0,0,0,0


In [187]:
df_unido = pd.merge(df_features, df_target_final, on ='ID', how = 'inner')
df_unido
#Ahora he logrado tener un solo dataframe en el que tengo tanto el ID de la molecula, luego sus features y al final los targets.

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,MaxPartialCharge,...,fr_ester,fr_para_hydroxylation,fr_phenol,fr_phenol_noOrthoHbond,hydroxamate,catecholate,a-hydroxycarboxylate,phenolate,citrate,other
ID,,,,,,,,,,,,,,,,,,,,,
6,14.637926,-6.027201,14.637926,1.541309,0.030202,782.889,724.425,782.406186,310,0.328347,...,1,0,0,0,3,0,0,0,0,0
6_ChEBI,14.204271,-5.464974,14.204271,1.515272,0.029440,726.825,672.393,726.379971,288,0.322416,...,1,0,0,0,3,0,0,0,0,0
7,14.235765,-5.588196,14.235765,1.524537,0.028291,740.852,684.404,740.395621,294,0.322629,...,1,0,0,0,3,0,0,0,0,0
11_r1,13.582822,-4.208434,13.582822,0.133729,0.577591,346.343,328.199,346.127720,132,0.293318,...,0,1,2,2,0,1,0,0,0,0
13,13.840994,-5.437481,13.840994,1.185498,0.052742,636.791,584.375,636.384663,254,0.249829,...,0,0,0,0,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,13.890962,-5.513798,13.890962,0.901957,0.037708,735.880,678.424,735.416691,292,0.245974,...,0,1,0,0,3,0,0,0,0,0
272,13.958666,-5.526200,13.958666,0.882605,0.037559,785.940,726.468,785.432341,310,0.245974,...,0,0,0,0,3,0,0,0,0,0
273,13.949972,-5.522786,13.949972,0.845568,0.037513,786.928,728.464,786.427590,310,0.245974,...,0,0,0,0,3,0,0,0,0,0


In [188]:
df_unido.reset_index(drop=True,inplace=True) #Para eliminar los indices que solia tener como nombre

In [189]:
from sklearn.multiclass import OneVsOneClassifier
features = df_unido.iloc[:,:-6]  #Mis features son todas las columnas del dataframe excepto las ultimas 6 columnas que corresponden a los targets
targets = df_unido.iloc[:, -6:]   #Mis targets son mis grupos funcionales o las clases en las que quiero configurar.
                                  #En mi nuevo dataframe mis targets son las ultimas 6 columnas
print(df_unido.columns)
features

Index(['MaxEStateIndex', 'MinEStateIndex', 'MaxAbsEStateIndex',
       'MinAbsEStateIndex', 'qed', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt',
       'NumValenceElectrons', 'MaxPartialCharge',
       ...
       'fr_ester', 'fr_para_hydroxylation', 'fr_phenol',
       'fr_phenol_noOrthoHbond', 'hydroxamate', 'catecholate',
       'a-hydroxycarboxylate', 'phenolate', 'citrate', 'other'],
      dtype='object', length=124)


,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,MaxPartialCharge,...,fr_NH0,fr_NH1,fr_NH2,fr_N_O,fr_amide,fr_benzene,fr_ester,fr_para_hydroxylation,fr_phenol,fr_phenol_noOrthoHbond
0,14.637926,-6.027201,14.637926,1.541309,0.030202,782.889,724.425,782.406186,310,0.328347,...,4,2,0,6,6,0,1,0,0,0
1,14.204271,-5.464974,14.204271,1.515272,0.029440,726.825,672.393,726.379971,288,0.322416,...,3,2,1,6,5,0,1,0,0,0
2,14.235765,-5.588196,14.235765,1.524537,0.028291,740.852,684.404,740.395621,294,0.322629,...,3,3,0,6,5,0,1,0,0,0
3,13.582822,-4.208434,13.582822,0.133729,0.577591,346.343,328.199,346.127720,132,0.293318,...,2,2,0,2,2,1,0,1,2,2
4,13.840994,-5.437481,13.840994,1.185498,0.052742,636.791,584.375,636.384663,254,0.249829,...,3,2,1,6,5,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,13.890962,-5.513798,13.890962,0.901957,0.037708,735.880,678.424,735.416691,292,0.245974,...,3,4,0,6,7,1,0,1,0,0
108,13.958666,-5.526200,13.958666,0.882605,0.037559,785.940,726.468,785.432341,310,0.245974,...,3,4,0,6,7,2,0,0,0,0
109,13.949972,-5.522786,13.949972,0.845568,0.037513,786.928,728.464,786.427590,310,0.245974,...,4,4,0,6,7,1,0,0,0,0
110,14.321259,-5.740742,14.321259,1.532044,0.027851,768.862,712.414,768.390536,304,0.328142,...,3,3,0,6,6,0,1,0,0,0


In [190]:
targets

,hydroxamate,catecholate,a-hydroxycarboxylate,phenolate,citrate,other
0,3,0,0,0,0,0
1,3,0,0,0,0,0
2,3,0,0,0,0,0
3,0,1,0,0,0,0
4,3,0,0,0,0,0
...,...,...,...,...,...,...
107,3,0,0,0,0,0
108,3,0,0,0,0,0
109,3,0,0,0,0,0
110,3,0,0,0,0,0


In [191]:
#Creo 6 modelos SVM para cada categoria (one vs. rest) debido a que tengo multiples categorias
modelos_SVM = []          #Creo un array para guardar todos los modelos clasificadores

for i in range(6):        #Creo una isntancia clasificadora para cada modelo con un kernel lineal (obtuve mejor resultado)
    svm_clasificador = SVC(kernel='linear')
    svm_clasificador.fit(X_train, y_train.iloc[:, i])  # Entreno el modelo con la i-ésima columna de mis targets
    modelos_SVM.append(svm_clasificador)  #Coloco cada modelos¿ clasificador

preddicion_SVM_modelos = []
for svm_clasificador in modelos_SVM:
    preddicion_SVM_modelos.append(svm_clasificador.predict(X_test)) #Predigo con el mismo set de datos de prueba para todos mis modelos entrenados

#3. Resultados

In [192]:
accuracies = []             #Calculo los accuracies modelo por modelo
for i in range(6):
    accuracy_i = accuracy_score(y_test.iloc[:, i], preddicion_SVM_modelos[i])
    accuracies.append(accuracy_i)
    print(f"Accuracy de la clasificación de moléculas con SVM para el modelo {i+1} {accuracy_i}")

Accuracy de la clasificación de moléculas con SVM para el modelo 1 0.782608695652174
Accuracy de la clasificación de moléculas con SVM para el modelo 2 0.9130434782608695
Accuracy de la clasificación de moléculas con SVM para el modelo 3 1.0
Accuracy de la clasificación de moléculas con SVM para el modelo 4 0.9565217391304348
Accuracy de la clasificación de moléculas con SVM para el modelo 5 1.0
Accuracy de la clasificación de moléculas con SVM para el modelo 6 1.0


#4. Discusión

Los valores parecen presentar overfitting debido a que la cantidad de datos que se usaron finalmente al filtrar las moleculas que solo pertenecian a una categoria excluyentemente culmino con un dataframe de menor cantidad de filas o moleculas

#5. Conclusiones

El preprocesamiento de la data es indispensable para pdoer realizar un entrenamiento y prediccion que se aproxime a la realidad de los datos, pues de lo contrario, la calidad de la data e informacion de la que se dispone quedaria subutilizada si no se le da un tratamiento adecuado.

Los kernels empleados para la clasificacion afectan de sobremanera los resultados que se pueden obtener en las metricas.



#6. Referencias

[1] Sideróforos microbianos, https://fciencias.ugr.es/images/stories/documentos/tfg_2223/biotecnologia/MIC-01.pdf (accessed Apr. 15, 2024).

[2] El papel de los sideróforos en las interacciones bacteria-bacteria y bacteria-hospedero, https://genomica.fciencias.unam.mx/2020/06/26/el-papel-de-los-sideroforos-en-las-interacciones-bacteria-bacteria-y-bacteria-hospedero/